In [4]:
!wget 'https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G' -O train_case2.csv

--2022-06-09 20:39:47--  https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G
Resolving drive.google.com (drive.google.com)... 173.194.205.101, 173.194.205.102, 173.194.205.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.205.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/idobbgha7eo8od460f4db39cdlm8ou4p/1654796325000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download [following]
--2022-06-09 20:39:49--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/idobbgha7eo8od460f4db39cdlm8ou4p/1654796325000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download
Resolving doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.com)... 172.217.169.161
Connecting to doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split

from lib.pipelines import CVDPipeline

In [2]:
df = pd.read_csv('train_case2.csv', sep=';')
df.head(5)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'),
                                                    df['cardio'], random_state=0)

In [4]:
params = {
    "continuos_cols" : ['age', 'height', 'weight', 'ap_hi', 'ap_lo'],
    "cat_cols" : ['gender', 'cholesterol'],
    "base_cols" : ['gluc', 'smoke', 'alco', 'active']
}

In [5]:
#logistic regression
lr_pipeline = CVDPipeline(params=params)
lr_pipeline.fit(X_train, y_train)
lr_pipeline.predict(X_test, y_test)

In [6]:
rf_pipeline = CVDPipeline(params=params)
rf_pipeline.fit(X_train, y_train, model_type="random forest")
rf_pipeline.predict(X_test, y_test)

In [7]:
gb_pipeline = CVDPipeline(params=params)
gb_pipeline.fit(X_train, y_train, model_type="gradient boosting")
gb_pipeline.predict(X_test, y_test)

In [8]:
data = [lr_pipeline.report, rf_pipeline.report, gb_pipeline.report]
df = pd.DataFrame.from_records(data)

In [9]:
ls = [lr_pipeline.estimator.cnf_matrix,
      rf_pipeline.estimator.cnf_matrix,
      gb_pipeline.estimator.cnf_matrix]

rubl_test = 1400
rubl_early_treatment = 15000
rubl_late_treatment = 20000

res = []
for i in ls:
    TN = i[0][0]
    FN = i[1][0]
    TP = i[1][1]
    FP = i[0][1]
    res.append( (FP + TP) * rubl_test + FN * rubl_late_treatment + TP * rubl_early_treatment)
df["costs, ₽"] = res



In [10]:
df

,model,thresh,F-Score,Precision,Recall,ROC AUC,"costs, ₽"
0,LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035,152974200
1,RandomForestClassifier,0.350000,0.718863,0.642669,0.815553,0.771037,153699400
2,LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035,152974200
